In [33]:
# Chapter1
# NN構築試行
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score  # モデルの評価を行うための関数
from keras import metrics
# 学習データ読み込み
data_X = pd.read_csv("train.csv")
data = data_X
data.head()

,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,loan_status
0,0,609.296068,3 years,8.421982,A5,0 years,debt_consolidation,714.061803,Individual,FullyPaid
1,1,1183.266999,5 years,10.286776,B1,10 years,credit_card,697.706701,Individual,ChargedOff
2,2,695.783256,3 years,14.723425,C2,1 year,debt_consolidation,656.419357,Individual,FullyPaid
3,3,738.392546,3 years,14.260708,C1,0 years,credit_card,657.906852,Individual,FullyPaid
4,4,1642.400654,5 years,25.217452,E5,10 years,debt_consolidation,662.972297,Individual,FullyPaid


In [34]:
# # Chapter2
# 学習の再現を行うために乱数シードを固定するためのコード
import os
import random as rn
from tensorflow.compat.v1.keras import backend as K

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(0)
rn.seed(0)

tf.compat.v1.set_random_seed(0)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph())
K.set_session(sess)

pd.set_option("max_columns", None)
# pd.set_option('max_rows', None)

In [35]:
# print("デフォルトした人:", len(data[data["loan_status"]=="ChargedOff"]))
# print("デフォルトしていない人:", len(data[data["loan_status"]=="FullyPaid"]))

In [36]:
# #Down Sampling
# Train1 = data.where(data["loan_status"] == "ChargedOff").dropna()
# Train2 = data.where(data["loan_status"] == "FullyPaid").dropna()
# Train3 = Train2.sample(n=len(Train1), random_state=0)
# TrainData = pd.concat([Train1, Train3])
# data = TrainData

In [37]:
data.head()

,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,loan_status
0,0,609.296068,3 years,8.421982,A5,0 years,debt_consolidation,714.061803,Individual,FullyPaid
1,1,1183.266999,5 years,10.286776,B1,10 years,credit_card,697.706701,Individual,ChargedOff
2,2,695.783256,3 years,14.723425,C2,1 year,debt_consolidation,656.419357,Individual,FullyPaid
3,3,738.392546,3 years,14.260708,C1,0 years,credit_card,657.906852,Individual,FullyPaid
4,4,1642.400654,5 years,25.217452,E5,10 years,debt_consolidation,662.972297,Individual,FullyPaid


In [38]:
# Chapter3
# def pre_process(input_data):
# データ前処理
# One-Hotエンコーディング
# pre_X = input_data.drop(["loan_status", "id"], axis=1).replace({
# IDは消す。グレード、勤続年数は順番付け。
# 登録方式は1/0。用途は「やばそうな理由」だけフラグ立て。
# pre_X = input_data.drop(["loan_status", "id"], axis=1).replace({
pre_X = data.drop(["loan_status","id"], axis=1).replace({
    "A1": 1, "A2": 2, "A3": 3, "A4": 4, "A5": 5,
    "B1": 6, "B2": 7, "B3": 8, "B4": 9, "B5": 10,
    "C1": 11, "C2": 12, "C3": 13, "C4": 14, "C5": 15,
    "D1": 16, "D2": 17, "D3": 18, "D4": 19, "D5": 20,
    "E1": 21, "E2": 22, "E3": 23, "E4": 24, "E5": 25,
    "F1": 26, "F2": 27, "F3": 28, "F4": 29, "F5": 30,
    "0 year": 0, "0 years": 0, "1 years": 1, "1 year": 1, "2 years": 2, "3 years": 3, "4 years": 4, "5 years": 5,
    "6 years": 6, "7 years": 7, "8 years": 8, "9 years": 9, "10 years": 10,
    "Individual": 1, "Joint App": 0,
    "purpose_car": 0,"purpose_credit_card": 1,"purpose_debt_consolidation": 1,"purpose_home_improvement": 0,"purpose_house": 0,"purpose_major_purchase": 0,"purpose_medical": 0,"purpose_other": 0,"purpose_small_business": 0,
}).astype({
    "term": "int8",
    "grade": "int8",
    "employment_length": "int8",
    "application_type": "int8",
})
pre_y = data["loan_status"]
X = pd.get_dummies(pre_X)
y = pre_y.replace({"ChargedOff": 1, "FullyPaid": 0})
# pre_X_train, pre_X_test = train_test_split(X, test_size=0.1, shuffle=False)
# y_train, y_test = train_test_split(y, test_size=0.1, shuffle=False)

# X_train = X
# X_test = X
y_train = y
y_test = y

# 加工が終わった時点でCSVに出すと、ほかのモデルでの使いまわしがきく。
# # Min-Maxスケーリング
# X_train = ((pre_X_train - pre_X_train.min()) / (pre_X_train.max() - pre_X_train.min()))
pre_X_train = ((X - X.min()) / (X.max() - X.min()))
# X_train = pre_X_train
# # Min-Maxスケーリング
# X_test = ((pre_X_test - pre_X_test.min()) / (pre_X_test.max() - pre_X_test.min()))
pre_X_test = ((X - X.min()) / (X.max() - X.min()))
# X_test = pre_X_test
    # return X_train, X_test, y_train, y_test
# X_train, X_test, y_train, y_test = pre_process(data)

In [39]:
pre_X_train.head()

,loan_amnt,term,interest_rate,grade,employment_length,credit_score,application_type,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_other,purpose_small_business
0,0.080922,0.0,0.121977,0.137931,0.0,0.382933,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.243609,1.0,0.205691,0.172414,1.0,0.276126,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.105436,0.0,0.404861,0.379310,0.1,0.006498,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.117513,0.0,0.384088,0.344828,0.0,0.016213,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.373746,1.0,0.875957,0.827586,1.0,0.049292,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
# 借入理由を債務整理のみに限定。フラグ系をINTに変換
pre_X_train = pre_X_train.astype("float32")
X_train = pre_X_train.drop(["purpose_home_improvement", "purpose_house", "purpose_major_purchase", "purpose_medical", "purpose_other", "purpose_small_business", "purpose_car","purpose_credit_card"], axis=1)

In [41]:
X_train.head()

,loan_amnt,term,interest_rate,grade,employment_length,credit_score,application_type,purpose_debt_consolidation
0,0.080922,0.0,0.121977,0.137931,0.0,0.382933,1.0,1.0
1,0.243609,1.0,0.205691,0.172414,1.0,0.276126,1.0,0.0
2,0.105436,0.0,0.404861,0.379310,0.1,0.006498,1.0,1.0
3,0.117513,0.0,0.384088,0.344828,0.0,0.016213,1.0,0.0
4,0.373746,1.0,0.875957,0.827586,1.0,0.049292,1.0,1.0


In [42]:
# SMOTE
from imblearn.over_sampling import SMOTE

sm = SMOTE(sampling_strategy='auto', k_neighbors=1, n_jobs=1, random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)
print("訓練データの特徴量", X_res.shape)
print("訓練データのターゲット", y_res.shape)

訓練データの特徴量 (399588, 8)
訓練データのターゲット (399588,)


In [43]:
# SMOTE加工前のデータ数
print("訓練データの特徴量", X_train.shape)
print("訓練データのターゲット", y_train.shape)


訓練データの特徴量 (242150, 8)
訓練データのターゲット (242150,)


In [44]:
from tensorflow import keras  # Keras
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.metrics import binary_accuracy, Accuracy
from tensorflow.keras.layers import SimpleRNN
import optuna


def create_model(units1, units2, lr):
    keras.backend.clear_session()
    # Keras以外を利用するのも一つの手段。
    model = keras.models.Sequential()
    model.add(Dense(units=units1, activation=tf.nn.relu)) 
    # model.add(Dropout(0.5))
    model.add(keras.layers.BatchNormalization())  # バッチ正規化層
    model.add(Dense(units=units2, activation=tf.nn.relu)) 
    # model.add(Dropout(0.5))
    model.add(keras.layers.BatchNormalization())  # バッチ正規化層
    model.add(Dense(units=2, activation=tf.nn.softmax))  # 出力層（活性化関数はソフトマックス関数）
    model.compile(optimizer = keras.optimizers.Adam(lr=0.01),
           loss="sparse_categorical_crossentropy",
            metrics="accuracy")

    return model

def objective(trial):
    keras.backend.clear_session()
    # 調整したいハイパーパラメータの設定
    # n_layer = trial.suggest_int('n_layer', 1, 2) # 追加する層を1-3から選ぶ
    units1 = int(trial.suggest_discrete_uniform('units1', 100, 500, 1)) # ユニット数
    units2 = int(trial.suggest_discrete_uniform('units2', 100, 500, 1)) # ユニット数
    lr = trial.suggest_float('lr', 0.01, 0.1,step=0.01) # 学習率
    

    model = create_model(units1, units2, lr)

    es_cb = keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 3)
    history = model.fit(x = X_res,
        y = y_res,
        batch_size = 8192, # バッチサイズ
        # epochs = 100, # エポック数
        epochs = 200, # エポック数
        validation_split = 0.1, # 検証データの割合
        # callbacks = [es_cb], # 早期終了の設定
        verbose = 1) # 進捗の確認を行うか（0:行わない, 1:行う）

    # return history.history["loss"][-1]
    # return history.history["val_loss"][-1]
    # return 1 - history.history["accuracy"][-1]
    return 1 - history.history["val_accuracy"][-1]



In [45]:
study = optuna.create_study()
study.optimize(objective, n_trials=1)

===========================] - 1s 23ms/step - loss: 0.6077 - accuracy: 0.6502 - val_loss: 0.7085 - val_accuracy: 0.5374
Epoch 64/200
44/44 [==============================] - 1s 24ms/step - loss: 0.6079 - accuracy: 0.6499 - val_loss: 0.6927 - val_accuracy: 0.5521
Epoch 65/200
44/44 [==============================] - 1s 24ms/step - loss: 0.6081 - accuracy: 0.6507 - val_loss: 0.6764 - val_accuracy: 0.6041
Epoch 66/200
44/44 [==============================] - 1s 23ms/step - loss: 0.6084 - accuracy: 0.6492 - val_loss: 0.7104 - val_accuracy: 0.5177
Epoch 67/200
44/44 [==============================] - 1s 24ms/step - loss: 0.6080 - accuracy: 0.6497 - val_loss: 0.6891 - val_accuracy: 0.5642
Epoch 68/200
44/44 [==============================] - 1s 23ms/step - loss: 0.6083 - accuracy: 0.6493 - val_loss: 0.7195 - val_accuracy: 0.5087
Epoch 69/200
44/44 [==============================] - 1s 23ms/step - loss: 0.6076 - accuracy: 0.6505 - val_loss: 0.6953 - val_accuracy: 0.5341
Epoch 70/200
44/44 [==

In [46]:
sorted_best_params = sorted(study.best_params.items(), key=lambda x : x[0])
best_param = dict()
for i, k in sorted_best_params:
    print(i + ' : ' + str(k))
    best_param[i] = k

lr : 0.09
units1 : 494.0
units2 : 322.0


In [47]:
keras.backend.clear_session()
# Keras以外を利用するのも一つの手段。
model = keras.models.Sequential()
model.add(Dense(units=best_param["units1"], activation=tf.nn.relu)) 
# model.add(Dropout(0.5))
model.add(keras.layers.BatchNormalization())  # バッチ正規化層
model.add(Dense(units=best_param["units2"], activation=tf.nn.relu)) 
# model.add(Dropout(0.5))
model.add(keras.layers.BatchNormalization())  # バッチ正規化層
model.add(Dense(units=2, activation=tf.nn.softmax))  # 出力層（活性化関数はソフトマックス関数）
model.compile(optimizer = keras.optimizers.SGD(lr=0.01),
        loss="sparse_categorical_crossentropy",
        metrics="accuracy")

In [48]:
history = model.fit(x = X_res,
        y = y_res,
        batch_size = 8192, # バッチサイズ
        # epochs = 100, # エポック数
        epochs = 200, # エポック数
        validation_split = 0.05, # 検証データの割合
        # callbacks = [es_cb], # 早期終了の設定
        verbose = 1) # 進捗の確認を行うか（0:行わない, 1:行う）

 val_accuracy: 0.6456
Epoch 62/200
47/47 [==============================] - 1s 23ms/step - loss: 0.6110 - accuracy: 0.6472 - val_loss: 0.6113 - val_accuracy: 0.6700
Epoch 63/200
47/47 [==============================] - 1s 23ms/step - loss: 0.6113 - accuracy: 0.6458 - val_loss: 0.6093 - val_accuracy: 0.6872
Epoch 64/200
47/47 [==============================] - 1s 23ms/step - loss: 0.6112 - accuracy: 0.6464 - val_loss: 0.7118 - val_accuracy: 0.5040
Epoch 65/200
47/47 [==============================] - 1s 23ms/step - loss: 0.6111 - accuracy: 0.6469 - val_loss: 0.6165 - val_accuracy: 0.6746
Epoch 66/200
47/47 [==============================] - 1s 23ms/step - loss: 0.6113 - accuracy: 0.6469 - val_loss: 0.6300 - val_accuracy: 0.6505
Epoch 67/200
47/47 [==============================] - 1s 23ms/step - loss: 0.6109 - accuracy: 0.6464 - val_loss: 0.6505 - val_accuracy: 0.6213
Epoch 68/200
47/47 [==============================] - 1s 23ms/step - loss: 0.6108 - accuracy: 0.6471 - val_loss: 0.6233 

In [49]:
# epoch数による正解率の推移を描画。
# ※大体200で均衡状態になるっぽいのでソースとしては用済み。

# import matplotlib.pyplot as plt
# #% matplotlib inline
# x = range(int(best_param["epochs"]))  # 演習D epoch数をx軸に指定する。range()の()のなかに、数字を入力してください。数字はepochsで指定した数と同じにしてください。
# plt.plot(x, history.history["accuracy"], c="red")
# plt.plot(x, history.history["val_accuracy"], c="blue")
# plt.title("accuracy")
# plt.show()

In [50]:
# 混同行列による評価
y_proba = model.predict(X_res)
y_pred = np.argmax(y_proba, axis=1)
confmat = confusion_matrix(y_res, y_pred)
print(confmat)

[[140015  59779]
 [ 83983 115811]]


In [51]:
# 正答率による評価
print("正解率:", accuracy_score(y_res, y_pred))

正解率: 0.640224431164099


In [52]:
print(pd.DataFrame({'y': y_res, 'y_pred': y_pred}).head())  # 実際のクラスと分類結果を上から5つだけ表示

   y  y_pred
0  0       0
1  1       0
2  0       0
3  0       0
4  0       1


In [53]:
# 重回帰
# 説明変数の重みを確認
from sklearn import linear_model  # 線形回帰を行うためのモジュール
# 変数の準備
# X = data_oh.drop(["loan_status_ChargedOff", "loan_status_FullyPaid"], axis=1)  # 説明変数の設定
# y = data_oh["loan_status_FullyPaid"]  # 目的変数の設定

# 学習
lr = linear_model.LinearRegression()  # 線形回帰モデルのインスタンスを作成
lr.fit(X_train, y_train)  # 回帰の実行

# 結果の確認
print("回帰係数:")
print(pd.DataFrame({"Name": X_train.columns,
                    "Coefficients": lr.coef_}).sort_values(by='Coefficients'))  # 回帰係数
print("切片:", lr.intercept_)  # 切片
print("決定係数:", lr.score(X_train, y_train))  # 決定係数

回帰係数:
                         Name  Coefficients
3                       grade     -0.168507
5                credit_score     -0.064070
4           employment_length      0.005284
7  purpose_debt_consolidation      0.018229
0                   loan_amnt      0.031944
6            application_type      0.040122
1                        term      0.045696
2               interest_rate      0.533368
切片: -0.025510445
決定係数: 0.07914761039345974


In [54]:
X_train.head()

,loan_amnt,term,interest_rate,grade,employment_length,credit_score,application_type,purpose_debt_consolidation
0,0.080922,0.0,0.121977,0.137931,0.0,0.382933,1.0,1.0
1,0.243609,1.0,0.205691,0.172414,1.0,0.276126,1.0,0.0
2,0.105436,0.0,0.404861,0.379310,0.1,0.006498,1.0,1.0
3,0.117513,0.0,0.384088,0.344828,0.0,0.016213,1.0,0.0
4,0.373746,1.0,0.875957,0.827586,1.0,0.049292,1.0,1.0


In [55]:
# # ランダムフォレスト
# # データ前処理
# # データ分割
# X = data.drop(["id", "term", "grade", "employment_length", "purpose", "application_type", "loan_status"], axis=1)
# y_train = data["loan_status"].values
# # # Min-Maxスケーリング
# X = ((X - X.min()) / (X.max() - X.min()))

In [56]:
from sklearn.ensemble import RandomForestClassifier  # ランダムフォレストを実行するためのクラス
# 学習
RFC = RandomForestClassifier(max_depth=3, random_state=1)  # ランダムフォレストのインスタンスを作成
RFC.fit(X_res, y_res)  # ランダムフォレストの学習

# 分類結果の確認
y_pred = RFC.predict(X_res)  # 分類結果
print(pd.DataFrame({'y': y_res, 'y_pred': y_pred}).head())  # 実際のクラスと分類結果を上から5つだけ表示

   y  y_pred
0  0       0
1  1       0
2  0       1
3  0       1
4  0       1


In [57]:
# コード例2
# モデルの精度（正解率）の確認
print('正解率:', RFC.score(X_res, y_res))

正解率: 0.6400417429952852


In [58]:
# 混同行列による評価
y_proba = RFC.predict(X_res)
confmat = confusion_matrix(y_res, y_proba)
print(confmat)

[[113760  86034]
 [ 57801 141993]]


In [59]:
# history.history["val_accuracy"]
np.savetxt("C:\work\AI\hoge.dat", y_pred)

In [60]:
# データ前処理
data_test = pd.read_csv("test.csv")
# Chapter3
# データ前処理
# One-Hotエンコーディング
# pre_X = input_data.drop(["loan_status", "id"], axis=1).replace({
# IDは消す。グレード、勤続年数は順番付け。
# 登録方式は1/0。用途は「やばそうな理由」だけフラグ立て。
pre_X = data_test.drop(["id"], axis=1).replace({
    "A1": 1, "A2": 2, "A3": 3, "A4": 4, "A5": 5,
    "B1": 6, "B2": 7, "B3": 8, "B4": 9, "B5": 10,
    "C1": 11, "C2": 12, "C3": 13, "C4": 14, "C5": 15,
    "D1": 16, "D2": 17, "D3": 18, "D4": 19, "D5": 20,
    "E1": 21, "E2": 22, "E3": 23, "E4": 24, "E5": 25,
    "F1": 26, "F2": 27, "F3": 28, "F4": 29, "F5": 30,
    "0 year": 0, "0 years": 0, "1 years": 1, "1 year": 1, "2 years": 2, "3 years": 3, "4 years": 4, "5 years": 5,
    "6 years": 6, "7 years": 7, "8 years": 8, "9 years": 9, "10 years": 10,
    "Individual": 1, "Joint App": 0,
    "purpose_car": 0,"purpose_credit_card": 1,"purpose_debt_consolidation": 1,"purpose_home_improvement": 0,"purpose_house": 0,"purpose_major_purchase": 0,"purpose_medical": 0,"purpose_other": 0,"purpose_small_business": 0,
}).astype({
    "term": "int8",
    "grade": "int8",
    "employment_length": "int8",
    "application_type": "int8",
})
X = pd.get_dummies(pre_X)

# 加工が終わった時点でCSVに出すと、ほかのモデルでの使いまわしがきく。
X_test = ((X - X.min()) / (X.max() - X.min()))

In [61]:
# 借入理由を債務整理のみに限定。フラグ系をINTに変換
X_test = X_test.astype("float32")
X_test = X_test.drop(["purpose_home_improvement", "purpose_house", "purpose_major_purchase", "purpose_medical", "purpose_other", "purpose_small_business", "purpose_car", "purpose_credit_card", "purpose_moving"], axis=1).astype({
    "term": "int8",
    "grade": "int8",
    "employment_length": "int8",
    "application_type": "int8",
    "purpose_debt_consolidation": "int8",
})

In [62]:
X_test.head()

,loan_amnt,term,interest_rate,grade,employment_length,credit_score,application_type,purpose_debt_consolidation
0,0.233748,0,0.338255,0,1,0.170125,1,0
1,0.550417,1,0.388391,0,1,0.081955,1,1
2,0.528645,0,0.117750,0,1,0.306207,1,0
3,0.048324,0,0.055643,0,0,0.484529,1,1
4,0.563678,0,0.271754,0,0,0.274790,1,1


In [63]:
y_pred = RFC.predict(X_test)
np.savetxt("C:\work\AI\hoge2.dat", y_pred)


In [64]:
y_proba = model.predict(X_test)
y_pred = np.argmax(y_proba, axis=1)
np.savetxt("C:\work\AI\hoge.dat", y_pred)